In [1]:
import sys
import os
import imp
import pickle as pickle
import numpy as np

from optparse import OptionParser
from demos.diffusion import ContaminantTransportModelCenter
from collections import Iterable
from paper.examples.diffusion_common import *
from vuq import *

In [2]:
def make_model():
    num_dim = 3
    log_prior = PDFCollection([UniformND(2),
                               MultivariateNormal([[-1.]])])
    y = load_upperlowercenters_diffusion_data()
    solver = ContaminantTransportModelCenter()
    log_like = IsotropicGaussianLikelihood(y, solver)
    log_p = Joint(log_like, log_prior)

    return locals()

In [3]:
def parse_expr_callback(option, opt, value, parser):
    if value is not None:
        setattr(parser.values, option.dest, eval(value))


def parse_list_callback(option, opt, value, parser):
    if value is not None:
        value = [float(x) for x in value.split(',')]
        if len(value) == 0:
            value = value[0] 
    setattr(parser.values, option.dest, value)


def convert_to_list(l, n):
    if not isinstance(l, Iterable):
        l = [l] * n
    return l


def initialize_bounds(l, u, n):
    l = convert_to_list(l, n)
    u = convert_to_list(u, n)
    b = tuple((l[i], u[i]) for i in range(n))
    return b


def main(options):
    model = options.model
    log_p = model['log_p']
    log_prior = model['log_prior']
    comp = [MultivariateNormal(log_prior.sample().flatten())
            for i in range(options.num_comp)]
    log_q = MixtureOfMultivariateNormals(comp)
    if options.mu_init is not None:
        log_q.mu = options.mu_init
        print (str(log_q))
    entropy = eval(options.entropy_approximation + '()')
    expectation_functional = eval(options.expectation_functional + '(log_p)')
    elbo = EvidenceLowerBound(entropy, expectation_functional)
    optimizer = Optimizer(elbo)
    mu_bounds = initialize_bounds(options.mu_lower_bound, options.mu_upper_bound, log_q.num_dim)
    C_bounds = initialize_bounds(options.C_lower_bound, options.C_upper_bound, log_q.num_dim)
    print ('mu_bounds', mu_bounds)
    print ('C_bounds', C_bounds)
    output_file = options.output
    if output_file is None:
        output_file = os.path.abspath('2_corners_num_comp=' + str(options.num_comp) + '.pcl')
    if os.path.exists(output_file) and options.force:
        print ('-', output_file, 'exists')
        print ('- I am removing it')
        os.remove(output_file)
    if os.path.exists(output_file):
        print ('- I am not repeating the calculations')
        with open(output_file, 'rb') as fd:
            results = pickle.load(fd)
            L = results['L']
            log_q = results['log_q']
            nfev = results['nfev']
    else:
        L, nfev = optimizer.optimize(log_q,
                                     tol=options.tol,
                                     max_it=options.max_it,
                                     mu_bounds=mu_bounds,
                                     C_bounds=C_bounds,
                                     full_mu=options.optimize_full_mu)
        print (str(elbo))
        results = {}
        results['L'] = L
        results['log_q'] = log_q
        results['nfev'] = nfev
        with open(output_file, 'wb') as fd:
            pickle.dump(results, fd, protocol=pickle.HIGHEST_PROTOCOL)
    w = log_q.w
    mu = log_q.mu
    C = log_q.C
    c = np.vstack([np.diag(C[i, :, :]) for i in range(log_q.num_comp)])
    x_m = np.mean(w * mu, axis=0)
    samples = log_q.sample(options.std_samples)
    x_s = np.std(samples, axis=0)
    x_05 = np.percentile(samples, 5, axis=0)
    x_95 = np.percentile(samples, 95, axis=0)
    print ('{0:10s} {1:10s} {2:10s}'.format('Parameter', 'Mean', 'Std.'))
    print ('-' * 32)
    for i in range(log_q.num_dim):
        print ('{0:10s} {1:4.6f} +-{2:2.6f}'.format('x_' + str(i+1), x_m[i], 1.96 * x_s[i]))
    for i in range(log_q.num_dim):
        print ('(%1.3f, %1.3f)' % (x_05[i], x_95[i]))
    print ('Number of evaluations:', nfev)

In [4]:
def initialize_model(model_file, model_name='Model', comm=None):
    rank, size = get_rank_size(comm)
    print_once('Initializing the %s.\n' % model_name, comm=comm)
    print_once('-------------------------\n', comm=comm)
    for i in range(size):
        if rank == i:
            try:
                model = imp.load_source('', model_file)
            except Exception as e:
                signal_fatal_error('I couldn\'t load the %s.\n' % model_name,
                                   comm=comm, e=e)
            print_rank(i, 'initialized %s.\n' % model_name)
        mpi_wait(comm)
    print_once('Done.\n'
               '-------------------------\n', comm=comm)
    return model


def initialize_native_model(model_file, model_name='Native Model', comm=None):
    return initialize_model(model_file, model_name=model_name, comm=comm).make_model()

In [ ]:
from easydict import EasyDict as edict

options = edict({'model':make_model(),
                 'entropy_approximation': 'FirstOrderEntropyApproximation',
                 'expectation_functional': 'ThirdOrderExpectationFunctional',
                 'num_comp': 1,
                 'mu_lower_bound': [0] * 2 + [None],
                 'mu_upper_bound': [1] * 2 + [None],
                 'C_lower_bound': 1e-6,
                 'C_upper_bound': [10] * 3,
                 'max_it': 30,
                 'tol': 1e-2,
                 'std_samples': 1000,
                 'mu_init': None,
                 'output': None,
                 'force': None,
                 'optimize_full_mu': None})
main(options)

mu_bounds ((0, 1), (0, 1), (None, None))
C_bounds ((1e-06, 10), (1e-06, 10), (1e-06, 10))
mu [ 0.68602167  0.2905535  -2.80348462]
mu [ 0.73106494  0.39233056 -1.80969765]
mu [ 0.70160796  0.32577136 -2.45960508]
mu [ 0.70594465  0.32444711 -2.53043016]
mu [ 1.          0.17907804 -3.25380773]
mu [ 0.79641898  0.27972026 -2.75299746]
mu [ 1.          0.06381473 -3.73904051]
mu [ 0.85569832  0.21685224 -3.04011645]
mu [ 0.80639484  0.26914048 -2.80131545]
mu [ 1.          0.09341674 -3.48739585]
mu [ 0.86129866  0.21930759 -2.99587863]
mu [ 0.81436056  0.26191047 -2.82954364]
mu [ 1.         0.094628  -3.4791544]
mu [ 0.86629552  0.21511111 -3.01128039]


In [10]:
parser = OptionParser()
parser.add_option('-m', '--model', metavar='FILE', dest='model', default=make_model(),
                  help='specify the file containing the native model')
parser.add_option('--entropy-approximation', type=str, dest='entropy_approximation',
                  default='EntropyLowerBound',
                  help='specify the entropy approximation to be used')
parser.add_option('--expectation-functional', type=str, dest='expectation_functional',
                  default='ThirdOrderExpectationFunctional',
                  help='specify the expectation functional to be used')
parser.add_option('--num-comp', type=int, dest='num_comp', default=2,
                  help='specify the number of components that you want to use')
parser.add_option('--mu-lower-bound', type=str, dest='mu_lower_bound', action='callback', default=([0] * 2 + [None]) * 2,
                  callback=parse_expr_callback,
                  help='specify the lower bound for mu')
parser.add_option('--mu-upper-bound', type=str, dest='mu_upper_bound', default=([1] * 2 + [None]) * 2, action='callback',
                  callback=parse_expr_callback, 
                  help='specify the upper bound for mu') 
parser.add_option('--C-lower-bound', type=str, dest='C_lower_bound', action='callback',
                  callback=parse_expr_callback, default=1e-6,
                  help='specify the lower bound for C')
parser.add_option('--C-upper-bound', type=str, dest='C_upper_bound', action='callback',
                  callback=parse_expr_callback, default=[10]*3,
                  help='specify the upper bound for C')
parser.add_option('--max-it', type=int, dest='max_it', default=100,
                  help='specify the maximum number of iterations')
parser.add_option('--tol', type=float, dest='tol', default=1e-2,
                  help='specify the tolerance')
parser.add_option('--std-samples', type=int, dest='std_samples', default=10000,
                  help='specify the number of samples used in the estimation of the std')
parser.add_option('--optimize-full-mu', action='store_true', dest='optimize_full_mu',
                  help='optimize all the mus simultaneously')
parser.add_option('-f', '--force', action='store_true', dest='force',
                  help='force the calculation')
parser.add_option('-o', '--output', metavar='FILE', dest='output',
                  help='specify the output file')
parser.add_option('--mu-init', type=str, dest='mu_init', action='callback', default=np.array([[0.25, 0.75, -2.], [0.75, 0.75, -2.]]),
                  callback=parse_expr_callback,
                  help='specify the initial point for mu num_comp x num_dim')
options, args = parser.parse_args()
main(options)

Name: Mixture of Gaussians
Num Input: 3
Num comp: 2
Weights:
[0.5 0.5]
Comp 0
Name: Multivariate Normal
Num Input: 3
mu:
[ 0.25  0.75 -2.  ]
C:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
Comp 1
Name: Multivariate Normal
Num Input: 3
mu:
[ 0.75  0.75 -2.  ]
C:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

mu_bounds ((0, 1), (0, 1), (None, None))
C_bounds ((1e-06, 10), (1e-06, 10), (1e-06, 10))
mu [ 0.25  0.75 -2.  ]
mu [ 0.23912062  0.71736186 -1.00059198]
mu [ 0.21752091  0.67100744 -1.14883306]
mu [ 0.1311221   0.48558977 -1.74179737]
mu [ 0.          0.20419292 -2.64170238]
mu [ 0.0868912   0.39066728 -2.04535879]
mu [ 0.03912367  0.28815495 -2.37319245]
mu [ 0.06447963  0.34257054 -2.1991718 ]
mu [ 0.          0.         -4.09602851]
mu [ 0.04606074  0.24471377 -2.74101721]
mu [ 0.05588317  0.29689888 -2.4520616 ]
mu [ 0.15959933  0.         -6.38984904]
mu [ 0.08858797  0.20327782 -3.69376334]
mu [ 0.06951457  0.25787752 -2.96960421]
mu [ 0.128042    0.         -5.44612928]
mu [ 0.08756919  0.17

mu [ 0.71694639  0.22516651 -2.89200353]
mu [ 0.90080932  0.22528171 -2.88553613]
mu [ 0.90127123  0.225282   -2.88551988]
mu [ 0.90117881  0.22543385 -2.8855319 ]
mu [ 0.90072144  0.22617616 -2.88563862]
mu [ 0.90071756  0.22617567 -2.88568388]
mu [ 0.90070206  0.2261737  -2.88586495]
mu [ 0.90070145  0.2261709  -2.88637583]
mu [ 0.90072016  0.2261698  -2.88651522]
mu [ 0.9007216   0.22616948 -2.88651615]
      fun: -0.6765998078095681
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([-2.91490256e-05, -1.86445084e-05, -3.85169160e-06])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 11
      nit: 7
   status: 0
  success: True
        x: array([ 0.9007216 ,  0.22616948, -2.88651615])
c: [0.00085851 0.00135595 0.05016232 0.00091651 0.00142761 0.0530284 ]
c: [10.          9.91466349  0.31041302 10.          0.65928067  0.04390498]
c: [0.17944873 0.17841312 0.05481054 0.17950569 0.01317723 0.05286545]
c: [0.0041885  0.00465735 0.05024899 0

ValueError: operands could not be broadcast together with shapes (2,) (2,3) 

# Graphs

In [ ]:
from optparse import OptionParser
from paper.examples.diffusion_common import *
from paper.examples import diffusion_model
import matplotlib.pyplot as plt
import pickle as pickle
import tables as tb
import pandas as pd
from scipy import stats
from vuq import MultivariateNormal, MixtureOfMultivariateNormals


def main(options):
    data = load_diffusion_data()
    model = diffusion_model.make_model()
    model = model['solver']
    
    with open(options.var_file, 'rb') as fd:
        var_results = pickle.load(fd)
    log_q = var_results['log_q']

    w = log_q.w
    mu =log_q.mu
    C = log_q.C
    c = np.vstack([np.diag(C[i, :, :]) for i in range(log_q.num_comp)])
    
    x_m_var = np.mean(mu * w[:, None], axis=0)
    samples_var = log_q.sample(options.var_samples)
    x_s_var = np.std(samples_var, axis=0)
    
    for i in range(mu.shape[1]):
        fig = plt.figure( figsize=(20, 10))
        ax = fig.add_subplot(111)
        x_min = 0.001
        x_max = 0.3
        x_i = np.linspace(x_min, x_max, 100)
        xx_i = np.log(x_i)[:, None] if i == 2 else x_i[:, None]
        
        comp = [MultivariateNormal([[mu[j, i]]], C=[[c[j, i]]])
                for j in range(log_q.num_comp)]
        log_q_i = MixtureOfMultivariateNormals(comp)
        y_i = np.exp(log_q_i(xx_i)) / (x_i if i == 2 else 1.)
        ax.plot(x_i, y_i, 'r-', linewidth=2)
        
        if i == 2:
            ax.plot([0.05] * 100, np.linspace(0, 50, 100), 'g--', linewidth=2)
        elif i == 0:
            ax.plot([0.09] * 100, np.linspace(0, options.max_true_pxi, 100), 'g--', linewidth=2)
        else:
            ax.plot([0.23] * 100, np.linspace(0, options.max_true_pxi, 100), 'g--', linewidth=2)

        #ax.hist(samples_var[:, i], alpha=0.5, normed=True)
        name  = '\\xi_{%d}' % (i + 1) if i < 2 else '\sigma'
        xlabel = '$' + name + '$'
        ylabel = '$p(' + name + '|y)$'
        ax.set_xlabel(xlabel, fontsize=26)
        ax.set_ylabel(ylabel, fontsize=26)
        plt.setp(ax.get_xticklabels(), fontsize=26)
        plt.setp(ax.get_yticklabels(), fontsize=26)

        if i == 0:
            leg = plt.legend(['VAR ($L=%d$)' % log_q.num_comp, 'True value', 'MCMC (MALA)'], loc='best')
            plt.setp(leg.get_texts(), fontsize=26)
        if i == 2:
            leg = plt.legend(['VAR ($L=%d$)' % log_q.num_comp, 'True noise', 'MCMC (MALA)'], loc='best')
            plt.setp(leg.get_texts(), fontsize=26)
        if i == 5:
            ax.xaxis.set_major_locator(plt.MaxNLocator(6))
        plt.tight_layout()
        png_file = 'result_4_corners_input_' + str(i) + '.png'
        print ('- writing', png_file)
        plt.savefig(png_file)
        del fig

In [ ]:
from easydict import EasyDict as edict

options = edict({'var_file':'C:\\Users\\Fedor\\Documents\\GitHub\\variational-reformulation-of-inverse-problems\\pcl\\2_corners_num_comp=2.pcl',
                 'num_samples':10000,
                 's':100,
                 'thin':1,
                 'max':-1,
                 'var_samples':10000,
                 'rolling':100,
                 'max_true_pxi':50.})
main(options)